In [1]:
MIN_NUMBER_OF_QUBITS = 1
MAX_NUMBER_OF_QUBITS = 3
USE_SIMULATOR = True

In [4]:
# Ansatz Circuit Construction
import jaqalpaq as jql
from qscout.v1.std.jaqal_gates import ALL_GATES
import numpy as np

def _prepare_state(
    circuit_parameters,
    number_of_qubits,
    circuit_builder,
):
    """Constructs the circuit to prepare the initial state that is used during measurement of all cliques"""
    assert len(circuit_parameters) == number_of_qubits
    assert number_of_qubits > 0
    
    # The reduced-unarily-encoded EGO circuit
    qubits = circuit_builder.register("q", number_of_qubits)
    circuit_builder.gate("prepare_all")
        
    
    theta0 = circuit_builder.let("theta0", circuit_parameters[0])
    circuit_builder.gate("Ry", qubits[0], theta0)


    for control_qubit_index in range(0, number_of_qubits-1):
        control_qubit = qubits[control_qubit_index]
        target_qubit = qubits[control_qubit_index + 1]
        theta_value = circuit_parameters[control_qubit_index+1]
        
        positive_halved_theta = circuit_builder.let("h_theta{}".format(control_qubit_index+1), theta_value/2)
        negative_halved_theta = circuit_builder.let("nh_theta{}".format(control_qubit_index+1), -theta_value/2)
        
        circuit_builder = _add_controlled_ry_gate(circuit_builder, control_qubit, target_qubit,
                                                  positive_halved_theta, negative_halved_theta)
        
    return circuit_builder, qubits

def _add_cnot_gate(circuit_builder, control, target):
    """CNOT Gate from Maslov (2017)"""
    circuit_builder.gate("Sy", control)
    circuit_builder.gate("Sxx", control, target)
    circuit_builder.gate("Sxd", control)
    circuit_builder.gate("Sxd", target)
    circuit_builder.gate("Syd", control)
    return circuit_builder


def _add_hadamard_gate(circuit_builder, qubit):
    """Hadamard Gate from JAQAL (https://www.sandia.gov/quantum/Projects/quantum_assembly_spec.pdf)"""
    circuit_builder.gate("Sy", qubit)
    circuit_builder.gate("Px", qubit)
    return circuit_builder

def _add_controlled_ry_gate(circuit_builder, control, target,
                            positive_halved_theta, negative_halved_theta):
    #     Optimized Gate Decomposition for CRy
    circuit_builder.gate("Rz", target, -np.pi/2)
    circuit_builder.gate("Rx", target, positive_halved_theta)
    circuit_builder.gate("Rx", control, np.pi/2)
    circuit_builder.gate("Rz", control, np.pi/2)
    circuit_builder.gate("MS", control, target, 0, negative_halved_theta)
    circuit_builder.gate("Rz", target, np.pi/2)
    circuit_builder.gate("Rz", control, -np.pi/2)
    circuit_builder.gate("Rx", control, -np.pi/2)
    return circuit_builder

In [5]:
def create_clique1_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 1"""
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)

    builder, _ = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )
    builder.gate("measure_all")
    return builder.build()


def create_clique2_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 2"""
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)
    
    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )
    
    # Hadamard rotation on all qubits to change measurement context
    for i in range(number_of_qubits):
        builder = _add_hadamard_gate(builder, qubits[i])
    builder.gate("measure_all")
    return builder.build()


def create_clique3_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 3"""
    assert number_of_qubits > 1
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)

    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )

    builder = _add_hadamard_gate(builder, qubits[1])
    builder = _add_cnot_gate(builder, qubits[0], qubits[1])
    builder = _add_hadamard_gate(builder, qubits[0])

    builder.gate("measure_all")
    return builder.build()


def create_clique4_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 4"""
    assert number_of_qubits > 2
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)
    
    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )

    builder = _add_hadamard_gate(builder, qubits[2])
    builder = _add_cnot_gate(builder, qubits[1], qubits[2])
    builder = _add_hadamard_gate(builder, qubits[1])

    builder.gate("measure_all")
    return builder.build()

In [6]:
from jaqalpaq.generator import generate_jaqal_program

if USE_SIMULATOR:
    from qscout.v1.std.ionsim import IonSimErrorModel
    jql.core.result.ProbabilisticSubcircuit.CUTOFF_FAIL = 1e-4
    jql.core.result.ProbabilisticSubcircuit.CUTOFF_WARN = 1e-4


def get_bitstrings_from_measured_probability_distribution(
    number_of_qubits, number_of_samples, probability_distribution
):
    bitstrings = [
        format(i, "0{}b".format(number_of_qubits))[::-1]
        for i in range(2 ** number_of_qubits)]

    measurements = np.random.choice(bitstrings, size=number_of_samples, replace=True, p=probability_distribution)
    
    return measurements.tolist()

def get_batched_probabilities_for_clique_on_simulator(
    circuit_generator,
    circuit_parameters_list,
    number_of_qubits,
    number_of_samples, # unused, but here to keep function signature consistent
):
    # backend = IonSimErrorModel(
    #     number_of_qubits,
    #     model="standard",
    #     params=["dpower12", "dfreq1", "dphase1", "dtime"],
    #     v0={"dpower12": 5e-4, "dfreq1": 5e3, "dphase1": 5e-2, "dtime": 5e-3},
    #     sigmas={"dpower12": 5e-4, "dfreq1": 5e3, "dphase1": 5e-2, "dtime": 5e-3},
    # )
    all_probabilities = []
    for circuit_parameters in circuit_parameters_list:
        circuit = circuit_generator(circuit_parameters, number_of_qubits)
        print(generate_jaqal_program(circuit))
        sim_result = jql.emulator.run_jaqal_circuit(circuit)
        all_probabilities.append(list(sim_result.subcircuits[0].probability_by_int))
    return all_probabilities


def get_batched_probabilities_for_clique_on_hardware(
    circuit_generator,
    circuit_parameters_list,
    number_of_qubits,
    number_of_samples
):
    # Initialize ansatz circuit using placeholder (theta=0) for parameter values
    empty_parameters = [0 for _ in circuit_parameters_list[0]]
    circuit = circuit_generator(empty_parameters, number_of_qubits)
    
    # Construct jaqal code for circuit
    circuit_code = "\n".join(
        ["from UserPulseDefinitions.LaserRamseyGates usepulses *"]
        + [generate_jaqal_program(circuit)]
    )

    # Construct dictionary to use when batching circuits
    batch_dictionary = {"__repeats__": number_of_samples}
    
    batch_dictionary.update(
        {
            "theta{}".format(i): [
                parameters[i] for parameters in circuit_parameters_list
            ]
            for i, _ in enumerate(circuit_parameters_list[0])
        }
    )
    batch_dictionary.update(
        {
            "(1/2)theta{}".format(i).format(i): [
                parameters[i] / 2 for parameters in circuit_parameters_list
            ]
            for i, _ in enumerate(circuit_parameters_list[0])
        }
    )
    batch_dictionary.update(
        {
            "-(1/2)theta{}".format(i).format(i): [
                -parameters[i] / 2 for parameters in circuit_parameters_list
            ]
            for i, _ in enumerate(circuit_parameters_list[0])
        }
    )

    print("Executing a batch with {} circuits".format(len(circuit_parameters_list)))
    # Run code on device and record probabilities of measurement outcomes indexed by parameter values
    return [list(experiment.subcircuits[0].probability_by_int)
            for experiment in jql.run.run_jaqal_batch(circuit_code, batch_dictionary)]


In [32]:
# Circuit Parameter Values
# Ordered as:
#     M (number of qubits)
#         eigenstate
#             exact diagonalized energy
#             ansatz parameters for LMG EGO circuit
EIGENSTATE_PREPARATION_PARAMETERS = {
    1: {
        0: {
            "energy": -0.615769,
            "parameters": [3.85532],
        },
        1: {
            "energy": 2.02998,
            "parameters": [0.71373],
        },
    },
    2: {
        0: {
            "energy": -1.6164,
            "parameters": [3.28449, 3.73368],
        },
        1: {
            "energy": 1.54737,
            "parameters": [4.55842, 0.71480],
        },
        2: {
            "energy": 3.60457,
            "parameters": [4.85605, 0.42656],
        },
    },
    3: {
        0: {
            "energy": -2.62861,
            "parameters": [3.11251, 3.3066, 3.72502],
        },
        1: {
            "energy": 0.758044,
            "parameters": [3.55943, 1.95467, 0.70252],
        },
        2: {
            "energy": 3.21341,
            "parameters": [5.04419, 1.80616, 0.400646],
        },
    },
    4: {
        0: {
            "energy": -3.63551,
            "parameters": [3.14757, 3.09809, 3.31778, 3.72279],
        },
        1: {
            "energy": -0.215827,
            "parameters": [3.03327, 3.61185, 1.9595, 0.71284],
        },
        2: {
            "energy": 2.79821,
            "parameters": [3.93827, 1.5874, 4.5382, 0.37183],
        },
    },
    5: {
        0: {
            "energy": -4.63985,
            "parameters": [3.14036, 3.15256, 3.09041, 3.3243, 3.72197],
        },
        1: {
            "energy": -1.21344,
            "parameters": [3.16866, 2.98021, 3.6505, 1.95293, 0.72159],
        },
        2: {
            "energy": 1.98543,
            "parameters": [
                3.38361,
                2.34141,
                4.64321,
                1.80604,
                0.365988,
            ],
        },
    },
    6: {
        0: {
            "energy": -5.64284,
            "parameters": [3.14185, 3.13892, 3.15581, 3.08549, 3.32861, -2.56156],
        },
        1: {
            "energy": -2.21637,
            "parameters": [3.13503, 3.19157, 2.94893, 3.67573, 1.94643, -5.55489],
        },
        2: {
            "energy": 1.06339,
            "parameters": [3.07225, 3.47705, 2.29446, 4.62736, 1.86839, 0.364857],
        },
    },
    7: {
        0: {
            "energy": -6.64502,
            "parameters": [
                3.14154,
                3.14223,
                3.13779,
                3.15815,
                3.08202,
                3.33168,
                -2.56169,
            ],
        },
        1: {
            "energy": -3.22076,
            "parameters": [
                3.14315,
                3.12716,
                3.20743,
                2.92751,
                3.69362,
                1.94093,
                -5.54963,
            ],
        },
        2: {
            "energy": 0.104014,
            "parameters": [
                3.16061,
                3.01898,
                3.53528,
                2.26079,
                4.61856,
                1.91888,
                0.364886,
            ],
        },
    },
    8: {
        0: {
            "energy": -7.64668,
            "parameters": [
                3.1416,
                3.14144,
                3.14258,
                3.13688,
                3.15993,
                3.07943,
                3.33399,
                -2.56173,
            ],
        },
        1: {
            "energy": -4.22537,
            "parameters": [
                3.14123,
                3.14556,
                3.12072,
                3.2194,
                2.91171,
                3.70704,
                1.93636,
                -5.54541,
            ],
        },
        2: {
            "energy": -0.873359,
            "parameters": [
                3.13656,
                3.18228,
                2.98203,
                3.57583,
                2.23611,
                4.61235,
                1.95982,
                0.365333,
            ],
        },
    },
    9: {
        0: {
            "energy": -8.64799,
            "parameters": [
                3.14159,
                3.14163,
                3.14134,
                3.1429,
                3.13613,
                3.16133,
                3.07742,
                3.3358,
                -2.56172,
            ],
        },
        1: {
            "energy": -5.22978,
            "parameters": [
                3.14168,
                3.14054,
                3.14785,
                3.11534,
                3.22885,
                2.89948,
                3.71752,
                1.93255,
                -5.54196,
            ],
        },
        2: {
            "energy": -1.8605,
            "parameters": [
                3.14289,
                3.12899,
                3.19998,
                2.95389,
                3.60618,
                2.21727,
                4.60743,
                1.99355,
                0.365937,
            ],
        },
    },
    10: {
        0: {
            "energy": -9.64905,
            "parameters": [
                3.14159,
                3.14158,
                3.14166,
                3.14124,
                3.14317,
                3.13551,
                3.16247,
                3.07582,
                3.33725,
                -2.5617,
            ],
        },
        1: {
            "energy": -6.23384,
            "parameters": [
                3.14157,
                3.14186,
                3.13979,
                3.14996,
                3.11079,
                3.23653,
                2.88969,
                3.72594,
                1.92936,
                -5.53907,
            ],
        },
        2: {
            "energy": -2.85327,
            "parameters": [
                3.14127,
                3.1453,
                3.12179,
                3.21486,
                2.93149,
                3.62991,
                2.2024,
                4.60332,
                2.02178,
                0.366585,
            ],
        },
    },
    11: {
        0: {
            "energy": -10.6499,
            "parameters": [
                3.14159,
                3.14159,
                3.14158,
                3.14169,
                3.14115,
                3.14341,
                3.13499,
                3.16341,
                3.0745,
                3.33844,
                -2.56166,
            ],
        },
        1: {
            "energy": -7.23754,
            "parameters": [
                3.1416,
                3.14152,
                3.14209,
                3.13904,
                3.15187,
                3.1069,
                3.2429,
                2.88168,
                3.73287,
                1.92664,
                -5.53664,
            ],
        },
        2: {
            "energy": -3.84945,
            "parameters": [
                3.14167,
                3.14054,
                3.14793,
                3.11514,
                3.22756,
                2.91315,
                3.64906,
                2.19038,
                4.59976,
                2.04573,
                0.367228,
            ],
        },
    },
}

In [33]:
# Run LMG EGO Circuit to Prepare Ground States for Different Problem Sizes (Number of Qubits)
import json
import os
import datetime

date = datetime.datetime.now()

device = "hardware"
if USE_SIMULATOR:
    device = "simulator"
# RESULTS_DIRECTORY = "lmg-results/point/{}_{}-{}-{}/".format(device, date.month, date.day, date.year)
RESULTS_DIRECTORY = "debug/".format(device, date.month, date.day, date.year)

if not os.path.exists(RESULTS_DIRECTORY):
    os.makedirs(RESULTS_DIRECTORY)
    
NUMBER_OF_SAMPLES = 10000

for number_of_qubits in range(MIN_NUMBER_OF_QUBITS, MAX_NUMBER_OF_QUBITS+1):

    circuit_parameters = []
    # "0" chooses the ground-state to prepare, you can change this to "1" or "2" 
    #     to choose 1st or 2nd excited state respectively
    for parameter in EIGENSTATE_PREPARATION_PARAMETERS[number_of_qubits][0]["parameters"]:
        # Fit parameters to -pi -> pi range
        if parameter > np.pi:
            parameter -= 2*np.pi
        circuit_parameters.append(parameter)
    
    cliques_to_measure = [create_clique1_circuit, create_clique2_circuit]
    if number_of_qubits > 1:
        cliques_to_measure.append(create_clique3_circuit)
    if number_of_qubits > 2:
        cliques_to_measure.append(create_clique4_circuit)
    
    get_batched_probabilities_for_clique = get_batched_probabilities_for_clique_on_hardware
    if USE_SIMULATOR:
        get_batched_probabilities_for_clique = get_batched_probabilities_for_clique_on_simulator
    
    # Will return a 2D list indexed first by clique measurements, then each list item being the 
    #    measured probability distribution
    all_probability_measurements = [
        get_batched_probabilities_for_clique(
                clique,
                [circuit_parameters],
                number_of_qubits,
                NUMBER_OF_SAMPLES,
        )[0] for clique in cliques_to_measure
    ]
    
    data_filename = RESULTS_DIRECTORY + "probability_data_{}_qubits.json".format(number_of_qubits)
    with open(data_filename, "w") as f:
        f.write(json.dumps(all_probability_measurements))
    f.close()
    
    all_measurements = [
        get_bitstrings_from_measured_probability_distribution(
            number_of_qubits, NUMBER_OF_SAMPLES, probability_distribution
        )
        for probability_distribution in all_probability_measurements
    ]

    measurement_data_filename = RESULTS_DIRECTORY + "measurement_data_{}_qubits.json".format(number_of_qubits)
    
    with open(measurement_data_filename, "w") as f:
        f.write(json.dumps(all_measurements))
    f.close()

Circuit(usepulses=[], constants={'theta0': Constant('theta0', -2.4278653071795864)}, macros={}, native_gates={'prepare_all': BusyGateDefinition(prepare_all, []), 'R': GateDefinition(R, [Parameter('q', ParamType.QUBIT), Parameter('axis-angle', ParamType.FLOAT), Parameter('rotation-angle', ParamType.FLOAT)]), 'I_R': IdleGateDefinition(I_R, [Parameter('q', ParamType.QUBIT), Parameter('axis-angle', ParamType.FLOAT), Parameter('rotation-angle', ParamType.FLOAT)]), 'Rx': GateDefinition(Rx, [Parameter('q', ParamType.QUBIT), Parameter('angle', ParamType.FLOAT)]), 'I_Rx': IdleGateDefinition(I_Rx, [Parameter('q', ParamType.QUBIT), Parameter('angle', ParamType.FLOAT)]), 'Ry': GateDefinition(Ry, [Parameter('q', ParamType.QUBIT), Parameter('angle', ParamType.FLOAT)]), 'I_Ry': IdleGateDefinition(I_Ry, [Parameter('q', ParamType.QUBIT), Parameter('angle', ParamType.FLOAT)]), 'Rz': GateDefinition(Rz, [Parameter('q', ParamType.QUBIT), Parameter('angle', ParamType.FLOAT)]), 'I_Rz': IdleGateDefinition(I_R